# Energy Use Data Cleaning

# Introduction
This project is meant to gather insights on electricity usage.
The 1st steps needed are to clean up the raw data so, it's usable for visualizations and machine learning algorthms.

## Data Source
Data is from my energy company(ComEd) from the past year. 10_22_2022 to 10_22_2023
Data from the [My Green Button](https://secure.comed.com/MyAccount/MyBillUsage/pages/secure/GreenButtonConnectDownloadMyData.aspx) webpage on the ComEd website.

# Goals
* become familiar with the columns in the dataset
* remove redundant data
* clean anomalous data

# Column / header info
* **TYPE**: Electric usage
* **DATE**: Day recorded
* **START TIME**: start of recording in Hour:Minutes
* **END TIME**: end of recording in Hour:Minutes
* **USAGE**: Electric usage in kWh
* **UNITS**: Electric usage metric
* **COST**: amount charged for energy usage in USD
* **NOTES**: useless data

In [24]:
import pandas as pd
import numpy as np

In [25]:
# Import the energy use spreadsheet from the 'data' directory

# Define the directory path and the regular expression pattern
import glob
directory_path = "./data"
file_pattern = "energy_use*.csv"

# Use glob.glob to match filenames based on the pattern
file_path = glob.glob(f"{directory_path}/{file_pattern}")[0]
energy_df_raw = pd.read_csv(filepath_or_buffer=file_path, header=4)

In [26]:
print(energy_df_raw.columns)
energy_df_raw.head()

Index(['TYPE', 'DATE', 'START TIME', 'END TIME', 'USAGE', 'UNITS', 'COST',
       'NOTES'],
      dtype='object')


,TYPE,DATE,START TIME,END TIME,USAGE,UNITS,COST,NOTES
0,Electric usage,2022-10-22,00:00,00:29,0.11,kWh,$0.01,NaN
1,Electric usage,2022-10-22,00:30,00:59,0.13,kWh,$0.02,NaN
2,Electric usage,2022-10-22,01:00,01:29,0.09,kWh,$0.01,NaN
3,Electric usage,2022-10-22,01:30,01:59,0.20,kWh,$0.02,NaN
4,Electric usage,2022-10-22,02:00,02:29,0.10,kWh,$0.01,NaN


In [27]:
energy_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TYPE        17520 non-null  object 
 1   DATE        17520 non-null  object 
 2   START TIME  17520 non-null  object 
 3   END TIME    17520 non-null  object 
 4   USAGE       17520 non-null  float64
 5   UNITS       17520 non-null  object 
 6   COST        17520 non-null  object 
 7   NOTES       0 non-null      float64
dtypes: float64(2), object(6)
memory usage: 1.1+ MB


# Observations & TODOs
* column titles have a space in them, change to underscore
* start-end time intervals seem to be all the same
* 'TYPE', 'UNITS', 'NOTES' columns seem to have no variation in values.  aka can be removed.
* 'COST' is in text format instead of numeric
* 2 columns for date & time can be combined into datetime objs
* a usage duration column would likely simplify future data visulization/modeling

In [28]:
# copy raw data into a df to be cleaned
energy_df = energy_df_raw.copy()

In [29]:
# replace the spaces in column titles with underscores
energy_df.columns = energy_df.columns.str.replace(' ', '_')

In [30]:
# Check how homogenous are start & end times
print(energy_df['START_TIME'].unique())
print(energy_df['END_TIME'].unique())

['00:00' '00:30' '01:00' '01:30' '02:00' '02:30' '03:00' '03:30' '04:00'
 '04:30' '05:00' '05:30' '06:00' '06:30' '07:00' '07:30' '08:00' '08:30'
 '09:00' '09:30' '10:00' '10:30' '11:00' '11:30' '12:00' '12:30' '13:00'
 '13:30' '14:00' '14:30' '15:00' '15:30' '16:00' '16:30' '17:00' '17:30'
 '18:00' '18:30' '19:00' '19:30' '20:00' '20:30' '21:00' '21:30' '22:00'
 '22:30' '23:00' '23:30']
['00:29' '00:59' '01:29' '01:59' '02:29' '02:59' '03:29' '03:59' '04:29'
 '04:59' '05:29' '05:59' '06:29' '06:59' '07:29' '07:59' '08:29' '08:59'
 '09:29' '09:59' '10:29' '10:59' '11:29' '11:59' '12:29' '12:59' '13:29'
 '13:59' '14:29' '14:59' '15:29' '15:59' '16:29' '16:59' '17:29' '17:59'
 '18:29' '18:59' '19:29' '19:59' '20:29' '20:59' '21:29' '21:59' '22:29'
 '22:59' '23:29' '23:59']


## NOTE: Why start & end times have 0 variance are kept
I chose to keep start & end times because in future plans to generalize the project, there will likely be variance in these columns. This can cause problems with the API / Pipeline / generalizing the project and these columns could be useful in dealing with those problems.

In [31]:
# Printing all the unique values of uninteresting columns
print([energy_df['TYPE'].unique(),
    energy_df['UNITS'].unique(),
    energy_df['NOTES'].unique()])

[array(['Electric usage'], dtype=object), array(['kWh'], dtype=object), array([nan])]


In [32]:
# copy the raw df for cleaning
# dropping columns with 0 variance
energy_df = energy_df.drop(['TYPE', 'UNITS', 'NOTES'], axis='columns')
energy_df.head()

,DATE,START_TIME,END_TIME,USAGE,COST
0,2022-10-22,00:00,00:29,0.11,$0.01
1,2022-10-22,00:30,00:59,0.13,$0.02
2,2022-10-22,01:00,01:29,0.09,$0.01
3,2022-10-22,01:30,01:59,0.20,$0.02
4,2022-10-22,02:00,02:29,0.10,$0.01


In [33]:
# format the COST column to numeric
energy_df['COST'] = energy_df['COST'].str.replace('$', '', regex=True).astype(float)

## THOUGHTS: change COST to integer vs float?
It's probably better to format COST column as an integer representing the COST as an integer of pennies to avoid floating point rounding. This would require reformatting the USAGE column to be logically consistent and changing the kwh in the col description.

The imprecision loss from floating point rounding would likely be negligent especially considering there is already a precision loss from the penny cost rounding from the energy company probably does.

In [34]:
energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DATE        17520 non-null  object 
 1   START_TIME  17520 non-null  object 
 2   END_TIME    17520 non-null  object 
 3   USAGE       17520 non-null  float64
 4   COST        17520 non-null  float64
dtypes: float64(2), object(3)
memory usage: 684.5+ KB


In [35]:
# converted the START, END columns to datetime format
energy_df['DATE_TIME'] = energy_df['DATE'] + ' ' + energy_df['START_TIME']
energy_df['START_TIME'] = pd.to_datetime(energy_df['DATE_TIME'], infer_datetime_format=True)

energy_df['DATE_TIME'] = energy_df['DATE'] + ' ' + energy_df['END_TIME']
energy_df['END_TIME'] = pd.to_datetime(energy_df['DATE_TIME'], infer_datetime_format=True)
energy_df.drop(columns='DATE_TIME', inplace=True)

In [36]:
# convert the DATE to datetime format
energy_df['DATE'] = pd.to_datetime(energy_df['DATE'], infer_datetime_format=True)
energy_df.info()
energy_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DATE        17520 non-null  datetime64[ns]
 1   START_TIME  17520 non-null  datetime64[ns]
 2   END_TIME    17520 non-null  datetime64[ns]
 3   USAGE       17520 non-null  float64       
 4   COST        17520 non-null  float64       
dtypes: datetime64[ns](3), float64(2)
memory usage: 684.5 KB


,DATE,START_TIME,END_TIME,USAGE,COST
0,2022-10-22,2022-10-22 00:00:00,2022-10-22 00:29:00,0.11,0.01
1,2022-10-22,2022-10-22 00:30:00,2022-10-22 00:59:00,0.13,0.02
2,2022-10-22,2022-10-22 01:00:00,2022-10-22 01:29:00,0.09,0.01
3,2022-10-22,2022-10-22 01:30:00,2022-10-22 01:59:00,0.20,0.02
4,2022-10-22,2022-10-22 02:00:00,2022-10-22 02:29:00,0.10,0.01


In [37]:
# created USAGE Duration column from START & END times
energy_df['USAGE_DUR'] = energy_df['END_TIME'] - energy_df['START_TIME']

energy_df.head()

,DATE,START_TIME,END_TIME,USAGE,COST,USAGE_DUR
0,2022-10-22,2022-10-22 00:00:00,2022-10-22 00:29:00,0.11,0.01,0 days 00:29:00
1,2022-10-22,2022-10-22 00:30:00,2022-10-22 00:59:00,0.13,0.02,0 days 00:29:00
2,2022-10-22,2022-10-22 01:00:00,2022-10-22 01:29:00,0.09,0.01,0 days 00:29:00
3,2022-10-22,2022-10-22 01:30:00,2022-10-22 01:59:00,0.20,0.02,0 days 00:29:00
4,2022-10-22,2022-10-22 02:00:00,2022-10-22 02:29:00,0.10,0.01,0 days 00:29:00


In [38]:
# find any anomalies in the USAGE_DUR col
unique_durs = (energy_df['USAGE_DUR']).unique()
print(unique_durs)

[ 1740000000000 80940000000000]


In [39]:
weird_usage = energy_df[energy_df['USAGE_DUR']==unique_durs[1]]
weird_usage.head()

,DATE,START_TIME,END_TIME,USAGE,COST,USAGE_DUR
723,2022-11-06,2022-11-06 01:30:00,2022-11-06 23:59:00,2.03,0.26,0 days 22:29:00


In [40]:
# drop the row with weird start/end times, verify result with checking USAGE_DUR is homagenous
wu_start = weird_usage['START_TIME']
energy_df.set_index('START_TIME', inplace=True)
energy_df.drop(wu_start, inplace=True)
print((energy_df['USAGE_DUR']).unique())

[1740000000000]


In [41]:
# Output cleaned data to a csv
insert_i = 7
file_path_clean = file_path[:insert_i] + "clean_" + file_path[insert_i:]
print(file_path_clean)
energy_df.to_csv(file_path_clean)

./data/clean_energy_use_10_22_22_to_10_22_23.csv
